In [322]:
import numpy as np
import torch
import torch.nn as nn
import torch.utils.data
from torch.autograd import Variable
import matplotlib.pyplot as plt
%matplotlib inline

In [328]:
data = np.loadtxt('../data/concrete.txt',dtype=np.float32)
np.random.shuffle(data)

cutoff = int(len(data)*0.8)
X_train, Y_train  = data[:cutoff, :-1], data[:cutoff, -1]
X_test, Y_test = data[cutoff:, :-1], data[cutoff:, -1]

std_X_train = np.std(X_train, 0)
std_X_train[std_X_train == 0] = 1
mean_X_train = np.mean(X_train, 0)

X_train_norm = (X_train - np.full(X_train.shape, mean_X_train)) / np.full(X_train.shape, std_X_train)
X_test_norm = (X_test - np.full(X_test.shape, mean_X_train)) / np.full(X_test.shape, std_X_train)

mean_Y_train = np.mean(Y_train)
std_Y_train = np.std(Y_train)

Y_train_norm = (Y_train - mean_Y_train) / std_Y_train
Y_train_norm = np.array(Y_train_norm, ndmin = 2).T

Y_test_norm = (Y_test - mean_Y_train) / std_Y_train
Y_test_norm = np.array(Y_test_norm, ndmin = 2).T

In [329]:
model_params = {"learning_rate": 0.001,
                "dropout": 0.01,
                "hidden_width": 50,
                "batch_size": 32,
                "n_epoch": 40}

In [330]:
X_train_tensor = torch.from_numpy(X_train_norm)
Y_train_tensor = torch.from_numpy(Y_train_norm)
X_test_tensor  = torch.from_numpy(X_test_norm)
Y_test_tensor  = torch.from_numpy(Y_test_norm)

train_tensor_dataset = torch.utils.data.TensorDataset(X_train_tensor, Y_train_tensor)
test_tensor_dataset = torch.utils.data.TensorDataset(X_test_tensor, Y_test_tensor)

data_loader_train = torch.utils.data.DataLoader(dataset=train_tensor_dataset, 
                                          batch_size=128, 
                                          shuffle=True,
                                          num_workers=4)

data_loader_test = torch.utils.data.DataLoader(dataset=test_tensor_dataset, 
                                          batch_size=128, 
                                          shuffle=False,
                                          num_workers=4)

In [334]:
# Neural Network Model (1 hidden layer)
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)  
    
    def forward(self, x):
        x = F.dropout(x, p=0.01, training=True)
        x = self.fc1(x)
        x = self.relu(x)
        x = F.dropout(x, p=0.01, training=True)
        y = self.fc2(x)
        return y
    
model = Net(input_size=8, hidden_size=50, output_size=1)

In [335]:
loss_criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.1)#1.0778889384649698e-06)

In [336]:
# Train the Model
for epoch in range(40):
    for batch_index, (X, Y) in enumerate(data_loader_train):  
        # Convert torch tensor to Variable
        X_tensor = Variable(X).float()
        Y_tensor = Variable(Y).float()
        
        # Forward + Backward + Optimize
        optimizer.zero_grad()  # zero the gradient buffer
        predicted_Y = model(X_tensor)
        loss = loss_criterion(predicted_Y, Y_tensor)
        loss.backward()
        optimizer.step()
        
    if epoch % (10) == 0:
        print('Epoch {}, loss {}'.format(epoch,loss.data[0]))
print('Root mean square error: {}'.format(np.sqrt(np.mean(loss.data[0]))))

Epoch 0, loss 0.8742383122444153
Epoch 10, loss 0.5356131792068481
Epoch 20, loss 0.445757120847702
Epoch 30, loss 0.2821389138698578
Root mean square error: 0.5627486950737607


In [261]:
# Test the Model
mse = 0
for X, Y in data_loader_test:
    X_tensor = Variable(X).float()
    Y_tensor = Variable(Y).float()
    Y_predict = net(X_tensor)
    total += Y.size(0)
    #mse += Y_tensor.view(-1) - Y_predict.view(-1)

In [262]:
torch.sqrt(torch.mean(Y_tensor.view(-1) - Y_predict.view(-1)))

Variable containing:
 6.1953
[torch.FloatTensor of size 1]

In [294]:
v = []
for _ in range(100):
    v.append( net(torch.from_numpy(X_test_norm[1,:])).data.numpy().item() )

In [297]:
np.mean(v)

0.32891482293605806

In [299]:
np.std(v)

0.045830732844034723

1.0778889384649698e-06